In [3]:
%load_ext autoreload
%autoreload 2
from swin_transformer_v2_1D.model_parts import bcd_to_bdc,bdc_to_bcd
import torch

x = torch.rand(size=(8,239,4))
x.shape


torch.Size([8, 239, 4])

In [4]:
bcd_to_bdc(x).shape == (8,4,239)

True

In [5]:
bdc_to_bcd(bcd_to_bdc(x)).shape == x.shape

True

In [19]:
from swin_transformer_v2_1D.model_parts import unfold

x_bcd=bdc_to_bcd(x)
print(x_bcd.shape) #OK
input=x_bcd

window_size=3

_, channels, length = input.shape

# Unfold input
output: torch.Tensor = input.unfold(dimension=2, size=window_size, step=window_size)  #BxCxWindowsxWindowSize
        # .unfold(dimension=2, size=window_size, step=window_size)

output.shape

torch.Size([8, 4, 239])


torch.Size([8, 4, 79, 3])

In [20]:
# Reshape to [batch size * windows, channels, window size]
# output: torch.Tensor = output.permute(0, 2, 3, 1, 5, 4).reshape(-1, channels, window_size, window_size)

output: torch.Tensor = output.permute(0, 2, 1, 3).reshape(-1, channels, window_size)

# unfold(x_bcd,window_size=8).shape #should be (8*windows=632,4,3=window_size)
output.shape

torch.Size([632, 4, 3])

In [25]:
input=unfold(x_bcd,3)

In [26]:
input.shape

torch.Size([632, 4, 3])

In [28]:
# Get channels of windows
channels: int = input.shape[1]
length=239
# Get original batch size
batch_size: int = int(input.shape[0] // (length // window_size))
print(batch_size) #correct
# Reshape input to
output: torch.Tensor = input.view(batch_size, length // window_size, channels,
                                window_size)
output: torch.Tensor = output.permute(0, 2, 1, 3).reshape(batch_size, channels,length)
output.shape

8


RuntimeError: shape '[8, 4, 239]' is invalid for input of size 7584